In [61]:
!pip install pymorphy2
!pip install natasha
!pip install fake-useragent

In [72]:
import requests
from bs4 import BeautifulSoup
from sklearn.metrics import accuracy_score
import string
import time
import re
from collections import Counter
import nltk
nltk.download('punkt')
from time import sleep
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
session = requests.session()
headers = {'User-Agent': ua.random}

nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
def get_reviews(url): # функция для парсинга даты с кинопоиска
    time.sleep(5)
    reviews = []
    ua = UserAgent(verify_ssl=False)
    headers = {'User-Agent': ua.random}
    time.sleep(5)
    html = session.get(url, headers=headers)
    page = html.text
    soup = BeautifulSoup(page, 'html.parser')
    for i in soup.find_all('div', {'class':'brand_words'}):
        reviews.append(i.text)
    return reviews

In [46]:
def preproc(rev): # предобработка данных
    lemm_words = []
    rev = rev.lower()
    for i in '\n\r\t«»-…':
        rev = rev.replace(i, '')
    for i in string.punctuation:
        rev = rev.replace(i, '')
    words = [morph.parse(x)[0].normal_form for x 
             in nltk.word_tokenize(rev)] 
    for i in words:
        lemm_words.append(i)
    return lemm_words

In [47]:
def divide_data(good_reviews_raw, bad_reviews_raw): # разделение даты на трейнинг и тест
    good_train = []
    bad_train = []
    test_data = {}
    upd = {}
    for i in good_reviews_raw[:60]:
        for j in preproc(i):
            good_train.append(j)
    for i in bad_reviews_raw[:60]:
        for j in preproc(i):
            bad_train.append(j)
    for i in good_reviews_raw[61:71]:
        test_data[i] = 'positive'
    for i in bad_reviews_raw[61:71]:
        upd[i] = 'negative'
    test_data.update(upd)
    return good_train, bad_train, test_data

In [48]:
def get_word_set(reviews): # убираем стоп-слова и шум и делаем множество слов
    stop = []
    noise = []
    for i in Counter(stop).items():
      noise.append(i[0]) 
    return noise

In [49]:
def predict_review(review, good_words, bad_words): # предсказание
    score = {}
    score['positive'] = 0
    score['negative'] = 0
    for w in review:
        if w in good_words:
            score['positive'] += 1
        elif w in bad_words:
            score['negative'] += 1
    return Counter(score).most_common()

In [50]:
def only(good_words, bad_words): # делаем множества для хороших и плохих слов
    good_only = []
    bad_only = []
    for i in good_words:
        if i not in bad_words:
            good_only.append(i)
    for i in bad_words:
        if i not in good_words:
            bad_only.append(i)
    return good_only, bad_only

In [118]:
def test_model(test_data, good_only, bad_only): # тест модели
    gold = []
    results = []
    for i in test_data.items():
        results.append(predict_review(preproc(i[0]), good_only, bad_only)[0][0])
        gold.append(i[1])
    print("RESULTS:")
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [52]:
good_url = '''https://www.kinopoisk.ru/film/447301/reviews/ord/date/status/good/perpage/100/'''
bad_url = '''https://www.kinopoisk.ru/film/447301/reviews/ord/date/status/bad/perpage/100/'''
good_train, bad_train, test_data = divide_data(
    get_reviews(good_url), get_reviews(bad_url))
good_only, bad_only = only(
    get_word_set(good_train), get_word_set(bad_train))

In [53]:
test_model(test_data, good_only, bad_only)

RESULTS:
Accuracy: 1.0000


In [82]:
good_train

['более',
 'правильный',
 'перевод',
 'название',
 'это',
 'фильм',
 'быть',
 'бы',
 'всетаки',
 'внедрение',
 'внедрение',
 'идея',
 'в',
 'сон',
 'фильм',
 'безусловно',
 'красивый',
 'зрелищный',
 'захватывать',
 'и',
 'очень',
 'чувственный',
 'что',
 'не',
 'мудрено',
 'ведь',
 'действие',
 'разворачиваться',
 'на',
 'грань',
 'явь',
 'и',
 'сон',
 'и',
 'сам',
 'сюжет',
 'так',
 'закрутить',
 'что',
 'зритель',
 'не',
 'всегда',
 'понимать',
 'где',
 'он',
 'сейчас',
 'в',
 'реальность',
 'или',
 'в',
 'чьемтый',
 'сон',
 'а',
 'если',
 'в',
 'сон',
 'то',
 'в',
 'чей',
 'сон',
 'внутри',
 'сон',
 '4',
 'уровень',
 'погружение',
 'в',
 'сон',
 'читать',
 'в',
 'подсознание',
 'человек',
 'кроме',
 'тот',
 'сцена',
 'скакать',
 'из',
 'один',
 'уровень',
 'в',
 'другой',
 'так',
 'что',
 'близкий',
 'к',
 'конец',
 'фильм',
 'мозг',
 'уже',
 'мочь',
 'перестать',
 'понимать',
 'где',
 'он',
 'находиться',
 'так',
 'как',
 'терять',
 'контроль',
 'всё',
 'как',
 'в',
 'сценарий',
 

# Улучшения
### 1. Классификатор
Нужно будет обязательно сделать классификатор основанный на методах машинного обучения и векторизации слов, для лучшего распознавания также может помочь анализ не отдельных слов, а словосочетаний
### 2. Больше данных
Для лучшего распознавания (тем более для нейросети) нужно больше данных для обучения
### 3. Обход капчи
У кинопоиска капча и я не нашел API, можно найти способ обойти (ну это уже больше к парсингу)

# Обновление классификатора
Использовал Наташу как лучший теггер по результатам тестов, будем выделять группы не + глагол, так как есть различие между не понравился и понравился, прилагательное + существительное, так как может быть большой успех и большой провал, а также наречие + прилагательное, тк. очень плохой и очень хороший 
По идее это должно улучшить классификатор

In [73]:
def nts_normalized(tag):
  tag = re.sub('PROPN', 'NOUN', tag)
  tag = re.sub('ADP', 'PREP', tag)
  tag = re.sub('\wCONJ', 'CONJ', tag)
  tag = re.sub('AUX', 'VERB', tag)
  return tag

In [74]:
def make_pos(word_list):
  tags_list = []
  for w in word_list:
    doc = Doc(w)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    tags_list.append((nts_normalized(doc.tokens[0].pos)))
  return tags_list

In [79]:
def find_bigrams(words_list, tags_list):
  not_verb = []
  adj_noun = []
  adv_adj = []
  for i in range(len(words_list)):
    try:
      if words_list[i] == 'не' and tags_list[i+1] == 'VERB':
        not_verb.append(words_list[i] + ' ' + words_list[i+1])
      if tags_list[i] == 'ADJ' and tags_list[i+1] == 'NOUN':
        adj_noun.append(words_list[i] + ' ' + words_list[i+1])
      if tags_list[i] == 'ADV' and tags_list[i+1] == 'ADJ':
        adv_adj.append(words_list[i] + ' ' + words_list[i+1])
    except IndexError:
      pass
  return not_verb, adj_noun, adv_adj

In [89]:
def make_word_list(text):
  word_list = clean(text).replace('\n', ' ').split()
  return word_list
def clean(text):
  for i in punctuation:
    text = text.replace(i, '')
  return text.lower()
from string import punctuation

In [94]:
good_tags = make_pos(good_train)
bad_tags = make_pos(bad_train)
good_not_verb, good_adj_noun, good_adv_adj = find_bigrams(good_train, good_tags)
bad_not_verb, bad_adj_noun, bad_adv_adj = find_bigrams(bad_train, bad_tags)

In [109]:
good_only_bi, bad_only_bi = only(list(set(good_not_verb)) + list(set(good_adj_noun)) + list(set(good_adv_adj)),
                                 list(set(bad_not_verb)) + list(set(bad_adj_noun)) + list(set(bad_adv_adj)))

In [120]:
test_model(test_data, good_only_bi, bad_only_bi)

RESULTS:
Accuracy: 1.0000
